In [26]:
:dep itertools
:dep ndarray
:dep ndarray-interp
:dep vawt = {path="../"}
:dep csv
:dep ndarray-csv
:dep plotters = {default_features = false, features = ["evcxr", "all_series"]}

use std::fs::File;
use std::f64::consts::{FRAC_PI_2, PI};
use std::error::Error;

use itertools::Itertools;
use csv::ReaderBuilder;
use ndarray::{array, Array2, Axis, Array};
use ndarray_csv::Array2Reader;
use plotters::prelude::*;

use vawt::areofoil::Aerofoil;
use vawt::turbine::{VAWTSolver, Verbosity, TurbineSolution};

In [27]:
fn read_array(path: &str) -> Result<Array2<f64>, Box<dyn Error>> {
    let file = File::open(path)?;
    let mut reader = ReaderBuilder::new()
        .has_headers(false)
        .trim(csv::Trim::All)
        .delimiter(b',')
        .from_reader(file);

    let mut arr: Array2<f64> = reader.deserialize_array2_dynamic().unwrap();
    for mut datapoint in arr.axis_iter_mut(Axis(0)) {
        datapoint[0] = datapoint[0].to_radians();
    }
    Ok(arr)
}

In [24]:
let aerofoil = Aerofoil::builder()
    .add_data_row(read_array("NACA0018/NACA0018Re0080.data")?, 80_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0040.data")?, 40_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0160.data")?, 160_000.0)?
    .set_aspect_ratio(12.8)
    .update_aspect_ratio(true)
    .symmetric(true)
    .build()?;

In [33]:
{
let mut solver = VAWTSolver::new(&aerofoil);
solver.re(31_300.0)
    .solidity(0.3525)
    .n_streamtubes(72)
    .verbosity(Verbosity::Silent);
let solution = solver.tsr(3.25).solve_with_beta(0.0);

let n = 72;
let d_t_half = PI / n as f64;
let theta = Array::linspace(d_t_half, 2.0 * PI - d_t_half, n);

evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..2.0, 0f64..1f64)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}π", x))
        .draw()?;

    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.a(theta))), 
        &RED
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.c_tan(theta))), 
        &BLUE
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta / PI, solution.thrust_error(theta))),
        &GREEN
    ))?;
    Ok(())
})
}

0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
1.0
 
 
 
 
0.0π
 
 
 
0.5π
 
 
 
1.0π
 
 
 
1.5π
 
 
 
2.0π
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="44,345 52,386 60,376 69,366 77,352 85,335 94,325 102,311 110,301 119,290 127,283 135,280 143,277 152,280 160,283 168,290 177,297 185,307 193,318 202,325 210,328 218,335 227,342 235,345 243,352 252,355 260,359 268,359 277,362 285,362 293,366 302,376 310,390 318,407 327,424 335,440 343,440 351,424 360,400 368,373 376,345 385,314 393,287 401,266 410,246 418,235 426,222 435,211 443,198 451,187 460,177 468,167 476,160 485,150 493,139 501,126 510,119 518,112 526,109 535,105 543,112 551,122 559,139 568,160 576,184 584,215 593,239 601,280 609,318 618,345 626,362 634,139 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="44,440 52,440 60,440 69,439 77,435 85,431 94,427 102,424 110,420 119,417 127,413 135,410 143,407 152,404 160,402 168,401 177,400 185,401 193,401 202,402 210,402 218,403 227,403 235,404 243,404 252,405 260,405 268,405 277,405 285,406 293,411 302,418 310,426 318,435 327,440 335,440 343,440 351,440 360,439 368,437 376,436 385,436 393,436 401,435 410,435 418,434 426,435 435,435 443,437 451,438 460,439 468,440 476,440 485,440 493,440 501,440 510,440 518,440 526,440 535,440 543,440 551,440 559,440 568,440 576,440 584,440 593,440 601,440 609,440 618,440 626,440 634,440 "/>
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="44,440 52,436 60,440 69,437 77,434 85,440 94,436 102,440 110,439 119,440 127,440 135,440 143,440 152,437 160,438 168,439 177,440 185,440 193,437 202,437 210,440 218,440 227,438 235,440 243,437 252,438 260,438 268,440 277,437 285,436 293,440 302,440 310,440 318,435 327,440 335,440 343,440 351,440 360,440 368,440 376,440 385,440 393,440 401,438 410,440 418,439 426,440 435,434 443,435 451,440 460,438 468,437 476,440 485,440 493,434 501,440 510,440 518,440 526,440 535,440 543,440 551,440 559,432 568,440 576,437 584,433 593,440 601,440 609,440 618,440 626,434 634,440 "/>

In [30]:
{
    let mut solver = VAWTSolver::new(&aerofoil);
    solver.re(31_300.0)
        .solidity(0.3525)
        .n_streamtubes(72)
        .verbosity(Verbosity::Silent);
    let solution = solver.tsr(3.0).solve_with_beta(0.0);

    let tsr = Array::linspace(1.0,5.0,20);
    let mut cps = Vec::new();
    for &tsr in tsr.iter() {
        let solution = solver.tsr(tsr).solve_with_beta(0.0);
        let (ct, cp) = solution.ct_cp();
        cps.push(cp);
    }
    
    
    let theta = Array::linspace(0.0, 2.0 * PI, 100);
    evcxr_figure((640,480), |root| {    
        // Create a chart context with the specified ranges
        let mut chart = ChartBuilder::on(&root)
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(1f64..5f64, 0f64..1f64)?;
    
        // Customize the x-axis labels
        chart.configure_mesh()
            .x_labels(8)
            .y_labels(5)
            .draw()?;
    
        chart.draw_series(LineSeries::new(
            tsr.iter().cloned().zip(cps), 
            &RED
        ))?;
        Ok(())
    })
}

0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0

In [31]:
let x = Array::linspace(0.0, FRAC_PI_2, 100);
//let xy: impl Iter> = x.into_iter().map(|x|{aerofoil.cl_cd(x, 45_000.0)});
let arr = read_array("NACA0018/NACA0018Re0080.data")?;


evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..0.5, 0f64..2f64)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .x_label_formatter(&|x| format!("{:.1}π", x))
        .draw()?;

    chart.draw_series(LineSeries::new(x.iter().map( |&x| (x / PI, aerofoil.cl_cd(x, 101_725.0).cl())), &RED))?;
    chart.draw_series(LineSeries::new(x.iter().map( |&x| {
        let (cn, ct) = aerofoil.cl_cd(x, 101_725.0).to_tangential(x, 0.0);
        (x / PI, ct)
    }), &YELLOW))?;
    chart.draw_series(LineSeries::new(
        arr.axis_iter(Axis(0)).map(|a| (a[0] / PI, a[1])), &BLUE
    ))?;
    chart.draw_series(LineSeries::new(
        arr.axis_iter(Axis(0)).map(|a| (a[0] / PI, a[2])), &GREEN
    ))?;
    Ok(())
})

0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
0.0π
 
 
 
0.1π
 
 
 
0.2π
 
 
 
0.3π
 
 
 
0.4π
 
 
 
0.5π
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="40,439 46,423 52,405 58,387 64,370 70,353 76,337 82,322 88,308 94,297 100,290 106,288 112,290 118,293 124,295 130,297 136,298 142,299 148,299 154,299 161,298 167,298 173,297 179,296 185,295 191,294 197,293 203,292 209,290 215,289 221,288 227,287 233,286 239,285 245,284 251,283 257,282 263,281 269,281 275,280 282,279 288,279 294,279 300,279 306,278 312,279 318,279 324,279 330,279 336,280 342,281 348,281 354,282 360,283 366,284 372,286 378,287 384,288 390,290 396,292 403,294 409,296 415,298 421,300 427,303 433,305 439,308 445,310 451,313 457,316 463,319 469,322 475,326 481,329 487,332 493,336 499,340 505,343 511,347 517,351 524,355 530,359 536,363 542,367 548,372 554,376 560,380 566,385 572,389 578,393 584,398 590,403 596,407 602,412 608,416 614,421 620,426 626,430 632,435 639,439 "/>
<polyline fill="none" opacity="1" stroke="#FFFF00" stroke-width="1" points="40,440 46,440 52,440 58,440 64,439 70,437 76,435 82,432 88,429 94,426 100,423 106,421 112,421 118,420 124,421 130,421 136,421 142,421 148,421 154,421 161,422 167,422 173,422 179,422 185,422 191,423 197,423 203,423 209,423 215,424 221,424 227,424 233,424 239,425 245,425 251,425 257,425 263,426 269,426 275,426 282,427 288,427 294,427 300,428 306,428 312,428 318,428 324,429 330,429 336,429 342,430 348,430 354,430 360,431 366,431 372,431 378,432 384,432 390,432 396,433 403,433 409,433 415,433 421,434 427,434 433,434 439,435 445,435 451,435 457,435 463,436 469,436 475,436 481,436 487,437 493,437 499,437 505,437 511,437 517,438 524,438 530,438 536,438 542,438 548,438 554,439 560,439 566,439 572,439 578,439 584,439 590,439 596,439 602,439 608,439 614,439 620,439 626,439 632,439 639,439 "/>

In [32]:
evcxr_figure((640 * 2, 480), |root| {

    let root = root.titled("Lift Coefficient", ("Arial", 20).into_font())?;
    
    let (left, right) = root.split_horizontally(640);

    for (pitch, yaw, area) in [(0.6, 0.3, left), ( PI / 2.0, 0.0, right)] {
        let mut chart = ChartBuilder::on(&area)
            .build_cartesian_3d(0.0..0.5, 0.0..2.0, 40_000.0..160_000.0)?;
        chart.with_projection(|mut p| {
            p.pitch = pitch; 
            p.yaw = yaw;
            p.scale = 0.7;
            p.into_matrix() // build the projection matrix
        });

        chart.configure_axes()
            .x_formatter(&|x: &f64| format!("{:.1}°", (x * PI).to_degrees()))
            .draw()?;
        
        let x = Array::linspace(0.0, PI/2.0, 50);
        let y  = Array::linspace(40_000.0, 160_000.0, 15);
        let series = x
            .into_iter()
            .tuple_windows::<(f64, f64)>()
            .cartesian_product(y.iter().cloned().tuple_windows::<(f64, f64)>())
            .map(|((x0, x1),(y0, y1))|{
                Polygon::new(vec![
                        (x0 / PI, aerofoil.cl_cd(x0,y0).cl(), y0),
                        (x0 / PI, aerofoil.cl_cd(x0,y1).cl(), y1),
                        (x1 / PI, aerofoil.cl_cd(x1,y1).cl(), y1),
                        (x1 / PI, aerofoil.cl_cd(x1,y0).cl(), y0),
                    ],
                    &HSLColor(240.0 / 360.0 - 240.0 / 360.0 * 6.5 * aerofoil.cl_cd(x0,y0).cl() / 5.0,1.0,0.7)
                )
            });

        chart.draw_series(series);
    }
    
    Ok(())
})

Lift Coefficient
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0°
 
 
 
18.0°
 
 
 
36.0°
 
 
 
54.0°
 
 
 
72.0°
 
 
 
90.0°
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
40000.0
 
 
 
60000.0
 
 
 
80000.0
 
 
 
100000.0
 
 
 
120000.0
 
 
 
140000.0
 
 
 
160000.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0°
 
 
 
18.0°
 
 
 
36.0°
 
 
 
54.0°
 
 
 
72.0°
 
 
 
90.0°
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
40000.0
 
 
 
60000.0
 
 
 
80000.0
 
 
 
100000.0
 
 
 
120000.0
 
 
 
140000.0
 
 
 
160000.0